### **Yuang Chen, Siraaj Singh Monga, Ella Ren, Greg Satchell Jr**

In [3]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 8)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

## **Predicting the Chances of a Stroke Report**

### **Introduction:**

Strokes, often referred to as brain attacks, occur when blood clots block the blood supply to part of the brain (ischemic stroke). They can also occur when blood vessels in the brain burst (hemorrhagic stroke). It is imperative that strokes are met with immediate treatment to prevent brain damage and death. Strokes remain the second leading cause of death and cause a majority of disabilities worldwide (Katan & Luft, 2018). Here in Canada, strokes have risen to an occurrence rate of 108,707 per year - or roughly one every five minutes (Stroke in Canada is on the rise, 2022).

With this in mind, our research question is the following: How effective are the variables age, bmi, and avg_glucose_level at correctly predicting whether or not an individual is at risk of suffering a stroke? We intend on creating a classification model that will address the predictive research question above.

Stroke Prediction Dataset, the title of the dataset we chose for this project, was adapted from the Kaggle data scientist fedesoriano. The dataset provides ten clinical features/variables for predicting strokes: gender, age, hypertension, heart_disease, ever_married, work_type, residence_type, avg_glucose_level, bmi, and smoking_status. We will only be using three out of the ten variables as we believe these are the most important factors when determining a chance of stroke. The link to the dataset can be found in the references tab.


### **Methods & Results:**

*put written method explanation here*

In [14]:
set.seed(1234)

url <- "https://raw.githubusercontent.com/ersq/Group-42-Group-Project/main/healthcare-dataset-stroke-data.csv"
stroke_data <- read_csv(url)

stroke_data

Rows: 5110 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): gender, ever_married, work_type, Residence_type, bmi, smoking_status
dbl (6): id, age, hypertension, heart_disease, avg_glucose_level, stroke

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
44873,Female,81,0,0,Yes,Self-employed,Urban,125.20,40,never smoked,0
19723,Female,35,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
37544,Male,51,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0
44679,Female,44,0,0,Yes,Govt_job,Urban,85.28,26.2,Unknown,0


### **Discussion:**

### **References (APA):**



Fedesoriano. (2021, January 26). *Stroke Prediction Dataset*. Kaggle. Retrieved April 6, 2023, from https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?resource=download 

Heart and Stroke Foundation of Canada. (2022, December 20). *Stroke in Canada is on the rise*. Heart&Stroke . Retrieved April 6, 2023, from https://www.heartandstroke.ca/what-we-do/media-centre/news-releases/stroke-in-canada-is-on-the-rise#:~:text=A%20new%20study%20funded%20by,as%20well%20as%20better%20prevention. 

Katan, M., & Luft, A. (2018, May 23). *Global Burden of Stroke*. PubMed®. Retrieved April 6, 2023, from https://pubmed.ncbi.nlm.nih.gov/29791947/#:~:text=Stroke%20is%20the%20second%20leading,%2D%20and%20middle%2Dincome%20countries. 


